In [1]:
import pandas as pd
import numpy as np

match = pd.read_csv('tier_mact_matches.csv')
nonmatch = pd.read_csv('tier_mact_nonmatch.csv')


match.rename(columns={'NAICS_x':'NAICS',
                      'NAICS_y':'NAICS_sub'}, inplace=True)


combined = pd.concat([match, nonmatch])


#replicate the entry based on its unit count
combined['Unit Count'].fillna(value=1, inplace=True)

total_count = pd.DataFrame(combined.values.repeat(combined['Unit Count'], axis=0),
                           columns=combined.columns)


#put capacity values all under one column, Capacity (mmBtu/hr)
total_count['Capacity (mmBtu/hr)'].fillna(value=total_count['AGGR_HIGH_HEAT_CAPACITY'],inplace=True)

total_count['Capacity (mmBtu/hr)'].fillna(value=total_count['INPUT_HEAT_CAPACITY'],inplace=True)

total_count = total_count[(total_count['Capacity (mmBtu/hr)']!=0)&
                          (total_count['Capacity (mmBtu/hr)']!=18644810)&
                          (total_count['Capacity (mmBtu/hr)']!=18614040)].reset_index(drop=True)


#put facility name/id, unit name/id, FIPS all under one column 
total_count['FACILITY_NAME'].fillna(value=total_count['FacilityID'],inplace=True)

total_count['UNIT_NAME'].fillna(value=total_count['UnitID'],inplace=True)

total_count['FIPS'].fillna(value=total_count['FIPS_m'],inplace=True)

#drop unnecessary columns
total_count.drop(columns=['AGGR_HIGH_HEAT_CAPACITY','AGGR_HIGH_HEAT_CAPACITY_UOM',
                          'INPUT_HEAT_CAPACITY','INPUT_HEAT_CAPACITY_UNIT',
                          'FacilityID','UnitID', 'FIPS_m', 'Zip',
                          'tierID','mactID','State','Num_Empl'],inplace=True)

#take most recent reporting year - for all data (capacity > 0)

total_count_cap = total_count.groupby(
    ['FACILITY_NAME','UNIT_NAME','FUEL_TYPE'],group_keys=False).apply(
    lambda x: x.sort_values('REPORTING_YEAR'))

total_count_cap = total_count_cap.drop_duplicates(subset=['FACILITY_ID','FACILITY_NAME',
                                                      'UNIT_NAME','FUEL_TYPE','NAICS'], 
                                              keep="last").reset_index(drop=True)

#take most recent reporting year - for data where energy consumption > 0
total_count_energy = total_count.groupby(['FACILITY_NAME','UNIT_NAME','FUEL_TYPE'],group_keys=False).apply(
    lambda x: x.sort_values('REPORTING_YEAR'))

total_count_energy = total_count_energy[(total_count_energy['ENERGY_COM_MMBtu'].notna()) &
                           (total_count_energy['ENERGY_COM_MMBtu']!='na')]

total_count_energy = total_count_energy[total_count_energy.ENERGY_COM_MMBtu.astype(float)>0]

total_count_energy = total_count_energy.drop_duplicates(subset=['FACILITY_ID','FACILITY_NAME',
                                                      'UNIT_NAME','FUEL_TYPE','NAICS'], 
                                              keep="last").reset_index(drop=True)



In [5]:
#save to file
total_count_cap.to_csv('total_tier_mact_cap.csv')
total_count_energy.to_csv('total_tier_mact_energy.csv')

In [ ]:
# check that energy consumption calculations make sense by checking if energy consumption
# divided by operating hours is less than capacity; if greater, then check energy com. calculations 
"""
total_count['Op Hours Per Year'].fillna(value=8760,inplace=True)

energy_check = total_count[(total_count['ENERGY_COM_MMBtu'].notna()) &
                           (total_count['ENERGY_COM_MMBtu']!='na')]

cap_lessthan_mmbtuhr = energy_check[(((energy_check['ENERGY_COM_MMBtu'].astype(float))/energy_check[
    'Op Hours Per Year'])>energy_check['Capacity (mmBtu/hr)'])].copy()

cap_lessthan_mmbtuhr.loc[:,'ENERGY_MMBtu_hr_check'] = \
    (cap_lessthan_mmbtuhr['ENERGY_COM_MMBtu'].astype(float))/cap_lessthan_mmbtuhr['Op Hours Per Year']
    
cap_lessthan_mmbtuhr.to_csv('capacity_lessthan_mmbtuperhr.csv')
"""